---
## PHASE 0: Setup & GPU Check

In [ ]:
# Check GPU availability
import torch
print(f"🖥️ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   ⚠️ No GPU found. Training will be slow on CPU.")

In [ ]:
# Install dependencies
!pip install -q yfinance pyts torch torchvision pillow matplotlib seaborn scikit-learn pandas numpy

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# GASF generation
from pyts.image import GramianAngularField
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ All packages imported. Device: {device}")

---
## PHASE 1: Data Collection

In [ ]:
# Training tickers (production watchlist - 48 diverse tickers)
TICKERS = [
    # Tech Giants & Semiconductors
    "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "TSLA", "META", "AMD", "NFLX", "INTC",
    # Indices & Commodities
    "SPY", "QQQ", "IWM", "DIA", "GLD", "SLV", "USO", "TLT",
    # Financials
    "JPM", "BAC", "GS", "MS", "V", "MA",
    # Healthcare
    "JNJ", "PFE", "UNH", "LLY",
    # Energy
    "XOM", "CVX", "COP",
    # Consumer
    "KO", "PEP", "MCD", "SBUX",
    # Media & Entertainment
    "DIS", "CMCSA",
    # Industrials
    "BA", "CAT", "DE",
    # Retail
    "WMT", "TGT", "COST",
    # Software & Cloud
    "CRM", "ADBE", "ORCL", "CSCO"
]

START_DATE = '2020-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"📥 Downloading data for {len(TICKERS)} tickers from production watchlist...")
print(f"   Period: {START_DATE} to {END_DATE}")
print(f"   Coverage: Tech, Indices, Financials, Healthcare, Energy, Consumer, Industrials\n")

In [ ]:
# Download daily data
data = {}

for ticker in TICKERS:
    try:
        df = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        
        if len(df) > 100:  # Need at least 100 days
            data[ticker] = df
            print(f"  ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"  ✗ {ticker}: {e}")

print(f"\n✅ Downloaded {len(data)} tickers")

---
## PHASE 2: GASF Image Generation

In [ ]:
def generate_gasf_image(prices, image_size=64, method='summation'):
    """
    Convert price time series to GASF image.
    
    Args:
        prices: 1D array of prices
        image_size: Output dimension (64x64 or 128x128)
        method: 'summation' (GASF) or 'difference' (GADF)
    
    Returns:
        GASF image (image_size x image_size)
    """
    # Normalize to [-1, 1] (required for GASF)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    prices_normalized = scaler.fit_transform(prices.reshape(-1, 1)).flatten()
    
    # Generate GASF
    gasf = GramianAngularField(image_size=image_size, method=method)
    gasf_image = gasf.fit_transform(prices_normalized.reshape(1, -1))
    
    return gasf_image[0]

print("✅ GASF generation function defined")

In [ ]:
def generate_multi_channel_gasf(df, window_size=30, image_size=64):
    """
    Generate 5-channel GASF: Open, High, Low, Close, Volume
    Like AlphaGo's multiple feature planes.
    
    Returns: (5, image_size, image_size) tensor
    """
    channels = []
    
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        prices = df[col].values[-window_size:]
        gasf = generate_gasf_image(prices, image_size=image_size)
        channels.append(gasf)
    
    # Stack channels (5, H, W)
    return np.stack(channels, axis=0)

print("✅ Multi-channel GASF function defined")

In [ ]:
# Generate GASF dataset
print("🧬 GENERATING GASF IMAGES...")
print("   This encodes temporal patterns as geometric structures\n")

gasf_dataset = []
WINDOW_SIZE = 30  # 30 days history
IMAGE_SIZE = 30  # Must be <= WINDOW_SIZE for GASF
HORIZON = 5  # 5 days forward prediction
THRESHOLD = 0.03  # 3% return threshold

print(f"⚙️ Configuration:")
print(f"   Window: {WINDOW_SIZE} days")
print(f"   Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   Horizon: {HORIZON} days")
print(f"   Threshold: ±{THRESHOLD*100:.1f}%\n")

if len(data) == 0:
    print("❌ ERROR: No data available! Run the data download cell first.")
else:
    for ticker in data.keys():
        df = data[ticker]
        
        # Check if enough data
        if len(df) < WINDOW_SIZE + HORIZON:
            print(f"  ⚠️ {ticker}: Skipped (only {len(df)} days, need {WINDOW_SIZE + HORIZON})")
            continue
        
        ticker_samples = 0
        errors = 0
        
        for i in range(WINDOW_SIZE, len(df) - HORIZON, 5):  # Every 5 days
            window_df = df.iloc[i-WINDOW_SIZE:i]
            
            # Calculate forward return (label)
            future_price = df.iloc[i + HORIZON]['Close']
            current_price = df.iloc[i]['Close']
            forward_return = (future_price - current_price) / current_price
            
            # Label: 0=SELL, 1=HOLD, 2=BUY
            if forward_return > THRESHOLD:
                label = 2  # BUY
            elif forward_return < -THRESHOLD:
                label = 0  # SELL
            else:
                label = 1  # HOLD
            
            try:
                # Generate 5-channel GASF
                gasf_img = generate_multi_channel_gasf(window_df, WINDOW_SIZE, IMAGE_SIZE)
                
                gasf_dataset.append({
                    'image': gasf_img,
                    'label': label,
                    'ticker': ticker,
                    'date': df.index[i],
                    'return': forward_return
                })
                ticker_samples += 1
            except Exception as e:
                errors += 1
                if errors == 1:  # Show first error only
                    print(f"  ⚠️ {ticker}: GASF generation error: {str(e)[:100]}")
        
        if ticker_samples > 0:
            print(f"  ✓ {ticker}: {ticker_samples} GASF images")
        elif errors > 0:
            print(f"  ✗ {ticker}: Failed ({errors} errors)")
    
    if len(gasf_dataset) > 0:
        print(f"\n✅ Generated {len(gasf_dataset)} GASF images (5 channels, {IMAGE_SIZE}x{IMAGE_SIZE} each)")
    else:
        print(f"\n❌ No GASF images generated! Check errors above.")

In [ ]:
# Show class distribution
if len(gasf_dataset) == 0:
    print("\n⚠️ ERROR: No GASF images generated!")
    print("\n🔍 DEBUGGING INFO:")
    print(f"   Tickers downloaded: {len(data)}")
    print(f"   WINDOW_SIZE: {WINDOW_SIZE}")
    print(f"   HORIZON: {HORIZON}")
    
    if len(data) > 0:
        sample_ticker = list(data.keys())[0]
        print(f"\n   Sample ticker '{sample_ticker}':")
        print(f"   - Total rows: {len(data[sample_ticker])}")
        print(f"   - Min rows needed: {WINDOW_SIZE + HORIZON}")
        print(f"   - Available samples: {max(0, len(data[sample_ticker]) - WINDOW_SIZE - HORIZON)}")
        
        print("\n💡 POSSIBLE CAUSES:")
        print("   1. Ticker data too short (need > 35 days)")
        print("   2. Data download failed")
        print("   3. GASF generation error (check previous cell for errors)")
        
        print("\n🔧 SOLUTIONS:")
        print("   1. Re-run data download cell")
        print("   2. Check if tickers have sufficient history")
        print("   3. Reduce WINDOW_SIZE or HORIZON if needed")
else:
    labels = [x['label'] for x in gasf_dataset]
    print(f"\n📊 Class Distribution:")
    print(f"   SELL (0): {labels.count(0)} ({labels.count(0)/len(labels)*100:.1f}%)")
    print(f"   HOLD (1): {labels.count(1)} ({labels.count(1)/len(labels)*100:.1f}%)")
    print(f"   BUY (2): {labels.count(2)} ({labels.count(2)/len(labels)*100:.1f}%)")
    
    print(f"\n✅ Dataset ready with {len(gasf_dataset)} samples")

In [ ]:
# Visualize GASF example
if len(gasf_dataset) == 0:
    print("⚠️ No GASF images to visualize. Generate dataset first.")
elif len(gasf_dataset) > 100:
    sample = gasf_dataset[100]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    label_names = ['SELL', 'HOLD', 'BUY']
    fig.suptitle(f"GASF Example: {sample['ticker']} - Label: {label_names[sample['label']]} ({sample['return']*100:.2f}% return)", fontsize=14)
    
    # Plot each channel
    channel_names = ['Open', 'High', 'Low', 'Close', 'Volume']
    for idx, (ax, name) in enumerate(zip(axes.flat[:5], channel_names)):
        im = ax.imshow(sample['image'][idx], cmap='RdYlGn', aspect='auto')
        ax.set_title(f'{name} GASF ({IMAGE_SIZE}x{IMAGE_SIZE})')
        ax.axis('off')
        plt.colorbar(im, ax=ax, fraction=0.046)
    
    # Close channel (what CNN sees)
    axes.flat[5].imshow(sample['image'][3], cmap='RdYlGn', aspect='auto')
    axes.flat[5].set_title(f'Close GASF (CNN Input)')
    axes.flat[5].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("📸 This is what the CNN will see - geometric patterns encoding temporal correlations")
    print(f"   Each pixel represents correlation between 2 time points in the {WINDOW_SIZE}-day window")
else:
    print(f"⚠️ Only {len(gasf_dataset)} samples generated. Need > 100 for visualization.")

---
## 🚀 ENHANCED VERSION: Multi-Indicator GASF

**Current Results:** 59.7% accuracy with 5 channels (OHLCV)

**Enhancement Strategy:**
- Add 8 technical indicators as extra channels → **13 total channels**
- Increase window size: 30 → 60 days (more pattern context)
- Learnable channel attention weights
- Expected improvement: +5-10% accuracy → **65-70% target**

In [ ]:
# Enhanced technical indicators
def calculate_indicators(df):
    """
    Calculate 8 technical indicators for multi-channel GASF.
    
    Returns dict with: SMA, EMA, BB_upper, BB_lower, VWAP, MACD, RSI, ATR
    """
    close = df['Close']
    high = df['High']
    low = df['Low']
    volume = df['Volume']
    
    indicators = {}
    
    # 1. Simple Moving Average (20-day)
    indicators['SMA'] = close.rolling(20).mean().fillna(close)
    
    # 2. Exponential Moving Average (20-day)
    indicators['EMA'] = close.ewm(span=20, adjust=False).mean()
    
    # 3-4. Bollinger Bands
    bb_sma = close.rolling(20).mean()
    bb_std = close.rolling(20).std()
    indicators['BB_UPPER'] = (bb_sma + 2 * bb_std).fillna(close)
    indicators['BB_LOWER'] = (bb_sma - 2 * bb_std).fillna(close)
    
    # 5. VWAP (Volume Weighted Average Price)
    indicators['VWAP'] = (close * volume).rolling(20).sum() / volume.rolling(20).sum()
    indicators['VWAP'] = indicators['VWAP'].fillna(close)
    
    # 6. MACD (12, 26, 9)
    ema_12 = close.ewm(span=12, adjust=False).mean()
    ema_26 = close.ewm(span=26, adjust=False).mean()
    indicators['MACD'] = ema_12 - ema_26
    
    # 7. RSI (14-day)
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    indicators['RSI'] = 100 - (100 / (1 + rs))
    indicators['RSI'] = indicators['RSI'].fillna(50)  # Neutral
    
    # 8. ATR (Average True Range - volatility)
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    indicators['ATR'] = tr.rolling(14).mean().fillna(tr)
    
    return indicators

print("✅ Enhanced indicator calculation function defined")

In [ ]:
# Enhanced multi-channel GASF with indicators
def generate_enhanced_gasf(df, window_size=60, image_size=60):
    """
    Generate 13-channel GASF:
    - 5 OHLCV channels
    - 8 technical indicator channels
    
    Returns: (13, image_size, image_size) tensor
    """
    channels = []
    
    # Calculate indicators
    indicators = calculate_indicators(df)
    
    # Get window data
    window_df = df.iloc[-window_size:]
    
    # OHLCV channels (5)
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        prices = window_df[col].values
        gasf = generate_gasf_image(prices, image_size=image_size)
        channels.append(gasf)
    
    # Indicator channels (8)
    for name in ['SMA', 'EMA', 'BB_UPPER', 'BB_LOWER', 'VWAP', 'MACD', 'RSI', 'ATR']:
        values = indicators[name].iloc[-window_size:].values
        gasf = generate_gasf_image(values, image_size=image_size)
        channels.append(gasf)
    
    # Stack all 13 channels
    return np.stack(channels, axis=0)

print("✅ Enhanced 13-channel GASF function defined")
print("   Channels: OHLCV (5) + SMA, EMA, BB, VWAP, MACD, RSI, ATR (8) = 13 total")

In [ ]:
# Generate enhanced GASF dataset
print("🧬 GENERATING ENHANCED GASF IMAGES (13 channels)...")
print("   60-day window with OHLCV + 8 technical indicators\n")

enhanced_gasf_dataset = []
ENHANCED_WINDOW = 60  # Double the context
ENHANCED_IMAGE_SIZE = 60  # Larger images
ENHANCED_HORIZON = 5
ENHANCED_THRESHOLD = 0.03

print(f"⚙️ Enhanced Configuration:")
print(f"   Window: {ENHANCED_WINDOW} days (2x original)")
print(f"   Image size: {ENHANCED_IMAGE_SIZE}x{ENHANCED_IMAGE_SIZE}")
print(f"   Channels: 13 (OHLCV + 8 indicators)")
print(f"   Horizon: {ENHANCED_HORIZON} days")
print(f"   Threshold: ±{ENHANCED_THRESHOLD*100:.1f}%\n")

if len(data) == 0:
    print("❌ ERROR: No data available! Run the data download cell first.")
else:
    for ticker in data.keys():
        df = data[ticker]
        
        # Need more data for 60-day window + indicators (need ~80 days minimum)
        min_needed = ENHANCED_WINDOW + 30  # Extra for indicator warmup
        if len(df) < min_needed + ENHANCED_HORIZON:
            print(f"  ⚠️ {ticker}: Skipped (only {len(df)} days, need {min_needed + ENHANCED_HORIZON})")
            continue
        
        ticker_samples = 0
        errors = 0
        
        # Start after indicator warmup period
        for i in range(min_needed, len(df) - ENHANCED_HORIZON, 5):
            window_df = df.iloc[i-ENHANCED_WINDOW:i]
            
            # Calculate forward return
            future_price = df.iloc[i + ENHANCED_HORIZON]['Close']
            current_price = df.iloc[i]['Close']
            forward_return = (future_price - current_price) / current_price
            
            # Label
            if forward_return > ENHANCED_THRESHOLD:
                label = 2  # BUY
            elif forward_return < -ENHANCED_THRESHOLD:
                label = 0  # SELL
            else:
                label = 1  # HOLD
            
            try:
                # Generate 13-channel GASF
                gasf_img = generate_enhanced_gasf(
                    df.iloc[:i],  # Full history for indicators
                    ENHANCED_WINDOW, 
                    ENHANCED_IMAGE_SIZE
                )
                
                enhanced_gasf_dataset.append({
                    'image': gasf_img,
                    'label': label,
                    'ticker': ticker,
                    'date': df.index[i],
                    'return': forward_return
                })
                ticker_samples += 1
            except Exception as e:
                errors += 1
                if errors == 1:
                    print(f"  ⚠️ {ticker}: Error: {str(e)[:80]}")
        
        if ticker_samples > 0:
            print(f"  ✓ {ticker}: {ticker_samples} enhanced GASF images")
        elif errors > 0:
            print(f"  ✗ {ticker}: Failed ({errors} errors)")
    
    if len(enhanced_gasf_dataset) > 0:
        print(f"\n✅ Generated {len(enhanced_gasf_dataset)} enhanced GASF images")
        print(f"   13 channels × {ENHANCED_IMAGE_SIZE}×{ENHANCED_IMAGE_SIZE} = {13*ENHANCED_IMAGE_SIZE*ENHANCED_IMAGE_SIZE:,} values per sample")
    else:
        print(f"\n❌ No enhanced GASF images generated!")

In [ ]:
# Show enhanced class distribution
if len(enhanced_gasf_dataset) == 0:
    print("\n⚠️ No enhanced GASF images. Check errors above.")
else:
    labels_enh = [x['label'] for x in enhanced_gasf_dataset]
    print(f"\n📊 Enhanced Dataset Class Distribution:")
    print(f"   SELL (0): {labels_enh.count(0)} ({labels_enh.count(0)/len(labels_enh)*100:.1f}%)")
    print(f"   HOLD (1): {labels_enh.count(1)} ({labels_enh.count(1)/len(labels_enh)*100:.1f}%)")
    print(f"   BUY (2): {labels_enh.count(2)} ({labels_enh.count(2)/len(labels_enh)*100:.1f}%)")
    print(f"\n✅ Enhanced dataset ready: {len(enhanced_gasf_dataset)} samples")
    print(f"   Memory: ~{len(enhanced_gasf_dataset) * 13 * 60 * 60 * 4 / 1e6:.1f} MB")

In [ ]:
# Enhanced AlphaGo network with channel attention
class EnhancedAlphaGoNet(nn.Module):
    """
    Enhanced dual-head network with:
    - 13-channel input (OHLCV + 8 indicators)
    - Learnable channel attention weights
    - Deeper feature extraction
    """
    def __init__(self, num_actions=3, input_channels=13):
        super(EnhancedAlphaGoNet, self).__init__()
        
        # Channel attention: Learn which indicators matter most
        self.channel_attention = nn.Sequential(
            nn.Linear(input_channels, input_channels * 2),
            nn.ReLU(),
            nn.Linear(input_channels * 2, input_channels),
            nn.Sigmoid()
        )
        
        # Modified ResNet-18 for 13 channels
        resnet = models.resnet18(pretrained=False)
        
        # First conv: 13 channels → 64 features
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Initialize with ImageNet weights (replicate for extra channels)
        resnet_pretrained = models.resnet18(pretrained=True)
        with torch.no_grad():
            # Copy RGB weights and replicate for extra channels
            weight = resnet_pretrained.conv1.weight
            self.conv1.weight[:, :3] = weight
            # Duplicate RGB pattern for remaining 10 channels
            for i in range(3, input_channels):
                self.conv1.weight[:, i:i+1] = weight[:, i % 3:i % 3 + 1]
        
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet blocks
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        self.avgpool = resnet.avgpool
        
        # POLICY HEAD (deeper)
        self.policy_fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_actions)
        )
        
        # VALUE HEAD (deeper)
        self.value_fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Tanh()
        )
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        # Channel attention: Weight each channel
        channel_weights = self.channel_attention(x.mean(dim=[2, 3]))  # (B, C)
        channel_weights = channel_weights.view(B, C, 1, 1)  # (B, C, 1, 1)
        x = x * channel_weights  # Apply learned weights
        
        # Shared backbone
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        features = torch.flatten(x, 1)
        
        # Dual outputs
        policy_logits = self.policy_fc(features)
        value = self.value_fc(features)
        
        return policy_logits, value

print("✅ Enhanced AlphaGo network with channel attention defined")
print("   Features: 13-channel input, learnable weights, deeper heads")

In [ ]:
# Initialize enhanced model
enhanced_model = EnhancedAlphaGoNet(num_actions=3, input_channels=13)
enhanced_model = enhanced_model.to(device)

print(f"🎮 Enhanced AlphaGo Trading Network")
print(f"   Device: {device}")
print(f"   Input: 13-channel GASF (OHLCV + 8 indicators)")
print(f"   Image size: {ENHANCED_IMAGE_SIZE}×{ENHANCED_IMAGE_SIZE}")
print(f"   Channel attention: Learnable weights per indicator")
print(f"   Policy head: 3-class (SELL/HOLD/BUY)")
print(f"   Value head: Regression (expected return)")
print(f"   Parameters: {sum(p.numel() for p in enhanced_model.parameters()):,}")
print(f"\n💡 The network will learn which indicators are most predictive!")

In [ ]:
# Prepare enhanced dataset and loaders
if len(enhanced_gasf_dataset) > 0:
    split_idx_enh = int(len(enhanced_gasf_dataset) * 0.8)
    train_data_enh = enhanced_gasf_dataset[:split_idx_enh]
    test_data_enh = enhanced_gasf_dataset[split_idx_enh:]
    
    train_dataset_enh = GASFDataset(train_data_enh)
    test_dataset_enh = GASFDataset(test_data_enh)
    
    # Reduce batch size for larger images (60x60 vs 30x30)
    train_loader_enh = DataLoader(train_dataset_enh, batch_size=16, shuffle=True, num_workers=2)
    test_loader_enh = DataLoader(test_dataset_enh, batch_size=16, shuffle=False, num_workers=2)
    
    print(f"📦 Enhanced DataLoaders:")
    print(f"   Train: {len(train_dataset_enh)} samples, {len(train_loader_enh)} batches")
    print(f"   Test: {len(test_dataset_enh)} samples, {len(test_loader_enh)} batches")
    print(f"   Batch size: 16 (reduced for larger images)")
else:
    print("❌ No enhanced dataset available. Generate it first.")

In [ ]:
# Train enhanced model
if len(enhanced_gasf_dataset) == 0:
    print("❌ No enhanced dataset. Generate it first (run previous cells).")
elif 'enhanced_model' not in locals():
    print("❌ Enhanced model not initialized. Run the model initialization cell first.")
elif 'train_loader_enh' not in locals():
    print("❌ DataLoaders not created. Run the dataloader cell first.")
else:
    print("\n🚀 TRAINING ENHANCED ALPHAGO NETWORK...")
    print("="*70)
    print("13 Channels: OHLCV + SMA + EMA + BB + VWAP + MACD + RSI + ATR")
    print("60-day window: 2x more context than baseline")
    print("Channel Attention: Network learns which indicators matter")
    print("="*70 + "\n")
    
    history_enh, best_acc_enh = train_alphago_network(
        enhanced_model,
        train_loader_enh,
        test_loader_enh,
        num_epochs=25,  # More epochs for complex model
        policy_weight=1.0,
        value_weight=0.5
    )
    
    print(f"\n🏆 ENHANCED MODEL RESULTS:")
    print(f"   Best Policy Accuracy: {best_acc_enh:.2f}%")
    print(f"   Baseline (5 channels): 59.71%")
    print(f"   Improvement: {best_acc_enh - 59.71:+.2f}%")
    
    if best_acc_enh > 65:
        print(f"\n✅ TARGET ACHIEVED! Enhanced model > 65% accuracy")
    elif best_acc_enh > 59.71:
        print(f"\n📈 IMPROVEMENT! Enhanced model beats baseline")
    else:
        print(f"\n⚠️ No improvement. Try different hyperparameters or indicators")

In [ ]:
# Visualize enhanced training curves
if 'history_enh' in locals():
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Training loss
    axes[0].plot(history_enh['train_loss'], label='Enhanced (13 ch)', linewidth=2)
    if 'history' in locals():
        axes[0].plot(history['train_loss'], label='Baseline (5 ch)', linewidth=2, alpha=0.7)
    axes[0].set_title('Training Loss', fontsize=14)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Policy accuracy
    axes[1].plot(history_enh['test_policy_acc'], color='green', label='Enhanced', linewidth=2)
    if 'history' in locals():
        axes[1].plot(history['test_policy_acc'], color='blue', label='Baseline', linewidth=2, alpha=0.7)
    axes[1].axhline(y=65, color='red', linestyle='--', label='Target (65%)')
    axes[1].set_title('Policy Accuracy', fontsize=14)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Value MSE
    axes[2].plot(history_enh['test_value_mse'], color='red', label='Enhanced', linewidth=2)
    if 'history' in locals():
        axes[2].plot(history['test_value_mse'], color='orange', label='Baseline', linewidth=2, alpha=0.7)
    axes[2].set_title('Value MSE', fontsize=14)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('MSE')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('enhanced_training_curves.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("📊 Enhanced training curves saved")
else:
    print("⚠️ Train enhanced model first")

In [ ]:
# Analyze channel attention weights
if 'enhanced_model' in locals():
    enhanced_model.eval()
    
    # Get channel attention weights for a sample batch
    sample_batch = next(iter(test_loader_enh))[0].to(device)
    with torch.no_grad():
        channel_weights = enhanced_model.channel_attention(sample_batch.mean(dim=[2, 3]))
        avg_weights = channel_weights.mean(dim=0).cpu().numpy()
    
    # Channel names
    channel_names = ['Open', 'High', 'Low', 'Close', 'Volume', 
                     'SMA', 'EMA', 'BB_Upper', 'BB_Lower', 'VWAP', 
                     'MACD', 'RSI', 'ATR']
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(channel_names, avg_weights, color='steelblue', alpha=0.8)
    ax.set_title('Learned Channel Importance Weights', fontsize=14, fontweight='bold')
    ax.set_xlabel('Channel')
    ax.set_ylabel('Attention Weight')
    ax.axhline(y=avg_weights.mean(), color='red', linestyle='--', label=f'Mean: {avg_weights.mean():.3f}')
    ax.legend()
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.savefig('channel_attention_weights.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Show top indicators
    top_indices = np.argsort(avg_weights)[::-1][:5]
    print("\n🏆 TOP 5 MOST IMPORTANT INDICATORS:")
    for i, idx in enumerate(top_indices, 1):
        print(f"   {i}. {channel_names[idx]}: {avg_weights[idx]:.3f}")
    
    print("\n💡 The network learned which indicators are most predictive!")
else:
    print("⚠️ Train enhanced model first")

---
## 📥 Download Enhanced Model

Download the enhanced model for integration with your numerical models.

In [ ]:
# Save enhanced model and results
if 'enhanced_model' in locals() and 'history_enh' in locals():
    import json
    
    # Save model
    torch.save(enhanced_model.state_dict(), 'enhanced_alphago_model.pth')
    
    # Save results
    results_enh = {
        'model_type': 'enhanced_13_channel',
        'best_policy_accuracy': best_acc_enh,
        'final_policy_accuracy': history_enh['test_policy_acc'][-1],
        'final_value_mse': history_enh['test_value_mse'][-1],
        'num_train_samples': len(train_dataset_enh),
        'num_test_samples': len(test_dataset_enh),
        'num_tickers': len(data),
        'window_size': ENHANCED_WINDOW,
        'image_size': ENHANCED_IMAGE_SIZE,
        'num_channels': 13,
        'channels': ['Open', 'High', 'Low', 'Close', 'Volume', 
                     'SMA', 'EMA', 'BB_Upper', 'BB_Lower', 'VWAP', 
                     'MACD', 'RSI', 'ATR'],
        'horizon': ENHANCED_HORIZON,
        'threshold': ENHANCED_THRESHOLD,
        'history': history_enh,
        'baseline_accuracy': 59.71,
        'improvement': best_acc_enh - 59.71
    }
    
    with open('enhanced_training_results.json', 'w') as f:
        json.dump(results_enh, f, indent=2)
    
    print("✅ Enhanced model saved!")
    print("\n📥 Download these files:")
    print("   1. enhanced_alphago_model.pth")
    print("   2. enhanced_training_results.json")
    print("   3. enhanced_training_curves.png")
    print("   4. channel_attention_weights.png")
    
    print(f"\n📊 FINAL COMPARISON:")
    print(f"   Baseline (5 ch, 30x30):  59.71%")
    print(f"   Enhanced (13 ch, 60x60): {best_acc_enh:.2f}%")
    print(f"   Improvement:             {best_acc_enh - 59.71:+.2f}%")
else:
    print("⚠️ Train enhanced model first")

---
## 🌍 REGIME-AWARE ENHANCEMENT: Context Intelligence

**Problem:** Current model sees individual charts in isolation
**Solution:** Add market context awareness

**What We'll Add:**
1. **Market Regime Detection** → Bull/Bear/Sideways (affects which patterns work)
2. **Sector Rotation Context** → Is this sector hot or cold?
3. **Relative Strength** → Is ticker underperforming sector? (buying opportunity)
4. **Entry Timing** → Avoid chasing pumps, catch dips before reversal
5. **Cross-Ticker Correlation** → How does this relate to SPY/sector?

**New Channels:**
- SPY correlation (market beta)
- Sector ETF correlation (XLK, XLF, XLE, etc.)
- Relative strength index (vs sector)
- Volume surge indicator
- Regime encoding (bull=1, bear=-1, sideways=0)

**Expected Improvement:** +10-15% accuracy by understanding CONTEXT

In [ ]:
# Define sector mappings
SECTOR_ETFS = {
    'XLK': ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'META', 'AMD', 'NFLX', 'INTC', 'CRM', 'ADBE', 'ORCL', 'CSCO'],  # Tech
    'XLF': ['JPM', 'BAC', 'GS', 'MS', 'V', 'MA'],  # Financials
    'XLE': ['XOM', 'CVX', 'COP'],  # Energy
    'XLV': ['JNJ', 'PFE', 'UNH', 'LLY'],  # Healthcare
    'XLP': ['KO', 'PEP', 'MCD', 'SBUX', 'WMT', 'TGT', 'COST'],  # Consumer
    'XLI': ['BA', 'CAT', 'DE'],  # Industrials
    'XLC': ['DIS', 'CMCSA'],  # Communication
}

# Download sector ETFs
print("📥 Downloading sector ETFs for context...")
sector_data = {}

for etf in SECTOR_ETFS.keys():
    try:
        df = yf.download(etf, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        sector_data[etf] = df
        print(f"  ✓ {etf}: {len(df)} days")
    except Exception as e:
        print(f"  ✗ {etf}: {e}")

# Download SPY for market context
if 'SPY' not in data:
    try:
        spy_df = yf.download('SPY', start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
        if isinstance(spy_df.columns, pd.MultiIndex):
            spy_df.columns = spy_df.columns.get_level_values(0)
        sector_data['SPY'] = spy_df
        print(f"  ✓ SPY: {len(spy_df)} days")
    except Exception as e:
        print(f"  ✗ SPY: {e}")
else:
    sector_data['SPY'] = data['SPY']

print(f"\n✅ Downloaded {len(sector_data)} sector/market ETFs for context")

In [ ]:
# Regime detection function
def detect_market_regime(df, window=20):
    """
    Detect market regime: Bull (1), Bear (-1), Sideways (0)
    
    Uses:
    - Trend: SMA(20) slope
    - Volatility: ATR
    - Momentum: RSI
    """
    close = df['Close']
    high = df['High']
    low = df['Low']
    
    # Trend: SMA slope
    sma = close.rolling(window).mean()
    trend = (sma - sma.shift(window)) / sma.shift(window)
    
    # Volatility: ATR
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(14).mean()
    atr_norm = atr / close  # Normalize by price
    
    # Momentum: RSI
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    
    # Regime logic
    regime = pd.Series(0, index=df.index)
    
    # Bull: Uptrend + high momentum + low volatility
    bull_mask = (trend > 0.02) & (rsi > 55) & (atr_norm < 0.03)
    regime.loc[bull_mask] = 1
    
    # Bear: Downtrend + low momentum + high volatility
    bear_mask = (trend < -0.02) & (rsi < 45) & (atr_norm > 0.03)
    regime.loc[bear_mask] = -1
    
    # Sideways: Everything else (default 0)
    
    return regime.fillna(method='ffill').fillna(0)

print("✅ Market regime detection function defined")
print("   Bull (1): Uptrend + momentum + low volatility")
print("   Bear (-1): Downtrend + fear + high volatility")
print("   Sideways (0): Choppy, no clear direction")

In [ ]:
# Context indicators function
def calculate_context_indicators(ticker, ticker_df, spy_df, sector_etf_df):
    """
    Calculate regime-aware context indicators:
    1. SPY correlation (market beta)
    2. Sector correlation (sector beta)
    3. Relative strength vs sector
    4. Volume surge indicator
    5. Pump detector (avoid chasing)
    6. Dip detector (buy opportunity)
    """
    context = {}
    
    close = ticker_df['Close']
    volume = ticker_df['Volume']
    
    # Align dates
    common_dates = ticker_df.index.intersection(spy_df.index)
    ticker_aligned = ticker_df.loc[common_dates]
    spy_aligned = spy_df.loc[common_dates]
    
    # 1. SPY Correlation (20-day rolling)
    ticker_returns = ticker_aligned['Close'].pct_change()
    spy_returns = spy_aligned['Close'].pct_change()
    context['SPY_CORR'] = ticker_returns.rolling(20).corr(spy_returns).fillna(0)
    
    # 2. Sector Correlation
    if sector_etf_df is not None:
        common_dates_sector = ticker_df.index.intersection(sector_etf_df.index)
        sector_aligned = sector_etf_df.loc[common_dates_sector]
        sector_returns = sector_aligned['Close'].pct_change()
        ticker_returns_sector = ticker_df.loc[common_dates_sector]['Close'].pct_change()
        context['SECTOR_CORR'] = ticker_returns_sector.rolling(20).corr(sector_returns).fillna(0)
    else:
        context['SECTOR_CORR'] = pd.Series(0, index=ticker_df.index)
    
    # Reindex to match ticker_df
    for key in ['SPY_CORR', 'SECTOR_CORR']:
        context[key] = context[key].reindex(ticker_df.index, fill_value=0)
    
    # 3. Relative Strength vs Sector (20-day)
    if sector_etf_df is not None:
        common_dates_rs = ticker_df.index.intersection(sector_etf_df.index)
        ticker_perf = ticker_df.loc[common_dates_rs]['Close'].pct_change(20)
        sector_perf = sector_etf_df.loc[common_dates_rs]['Close'].pct_change(20)
        rs = (ticker_perf - sector_perf).fillna(0)
        context['REL_STRENGTH'] = rs.reindex(ticker_df.index, fill_value=0)
    else:
        context['REL_STRENGTH'] = pd.Series(0, index=ticker_df.index)
    
    # 4. Volume Surge (vs 20-day average)
    avg_volume = volume.rolling(20).mean()
    context['VOL_SURGE'] = (volume / avg_volume - 1).fillna(0).clip(-2, 2)
    
    # 5. Pump Detector (avoid chasing - recent 5-day gain > 10%)
    recent_gain = close.pct_change(5)
    context['PUMP'] = (recent_gain > 0.10).astype(float)
    
    # 6. Dip Detector (buy opportunity - down 5%+ from 20-day high but RSI < 40)
    high_20 = close.rolling(20).max()
    drawdown = (close / high_20 - 1)
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rsi = 100 - (100 / (1 + gain / (loss + 1e-10)))
    context['DIP_OPP'] = ((drawdown < -0.05) & (rsi < 40)).astype(float)
    
    return context

print("✅ Context indicators function defined")
print("   6 new indicators: SPY corr, Sector corr, Rel strength, Vol surge, Pump, Dip")

In [ ]:
# Find sector for ticker
def get_sector_etf(ticker):
    """Map ticker to its sector ETF"""
    for etf, tickers in SECTOR_ETFS.items():
        if ticker in tickers:
            return etf
    return None

print("✅ Sector mapping function defined")

In [ ]:
# Regime-aware GASF generation
def generate_regime_aware_gasf(ticker, df, spy_df, sector_data, window_size=60, image_size=60):
    """
    Generate regime-aware 20-channel GASF:
    - 5 OHLCV channels
    - 8 technical indicator channels
    - 6 context channels (SPY corr, sector corr, rel strength, vol surge, pump, dip)
    - 1 regime channel (bull/bear/sideways)
    
    Returns: (20, image_size, image_size) tensor
    """
    channels = []
    
    # Calculate all indicators
    indicators = calculate_indicators(df)
    
    # Find sector ETF
    sector_etf = get_sector_etf(ticker)
    sector_etf_df = sector_data.get(sector_etf) if sector_etf else None
    
    # Calculate context indicators
    context = calculate_context_indicators(ticker, df, spy_df, sector_etf_df)
    
    # Detect regime
    regime = detect_market_regime(df)
    
    # Get window data
    window_df = df.iloc[-window_size:]
    
    # OHLCV channels (5)
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        prices = window_df[col].values
        gasf = generate_gasf_image(prices, image_size=image_size)
        channels.append(gasf)
    
    # Technical indicator channels (8)
    for name in ['SMA', 'EMA', 'BB_UPPER', 'BB_LOWER', 'VWAP', 'MACD', 'RSI', 'ATR']:
        values = indicators[name].iloc[-window_size:].values
        gasf = generate_gasf_image(values, image_size=image_size)
        channels.append(gasf)
    
    # Context channels (6)
    for name in ['SPY_CORR', 'SECTOR_CORR', 'REL_STRENGTH', 'VOL_SURGE', 'PUMP', 'DIP_OPP']:
        values = context[name].iloc[-window_size:].values
        gasf = generate_gasf_image(values, image_size=image_size)
        channels.append(gasf)
    
    # Regime channel (1)
    regime_values = regime.iloc[-window_size:].values
    gasf = generate_gasf_image(regime_values, image_size=image_size)
    channels.append(gasf)
    
    # Stack all 20 channels
    return np.stack(channels, axis=0)

print("✅ Regime-aware 20-channel GASF function defined")
print("   Channels: OHLCV (5) + Indicators (8) + Context (6) + Regime (1) = 20 total")
print("   🌍 NOW THE CNN SEES: Chart patterns + Market context + Sector rotation")

In [ ]:
# Generate regime-aware GASF dataset
print("🌍 GENERATING REGIME-AWARE GASF IMAGES (20 channels)...")
print("   60-day window with full market context\n")

regime_aware_dataset = []
REGIME_WINDOW = 60
REGIME_IMAGE_SIZE = 60
REGIME_HORIZON = 5
REGIME_THRESHOLD = 0.03

print(f"⚙️ Regime-Aware Configuration:")
print(f"   Window: {REGIME_WINDOW} days")
print(f"   Image size: {REGIME_IMAGE_SIZE}x{REGIME_IMAGE_SIZE}")
print(f"   Channels: 20 (OHLCV + Indicators + Context + Regime)")
print(f"   Context: SPY correlation, sector rotation, relative strength")
print(f"   Intelligence: Knows to avoid pumps, catch dips, understand regimes")
print(f"   Horizon: {REGIME_HORIZON} days\n")

if len(data) == 0 or len(sector_data) == 0:
    print("❌ ERROR: No data available! Run the data download cells first.")
else:
    spy_df = sector_data['SPY']
    
    for ticker in data.keys():
        df = data[ticker]
        
        # Need more data for context calculation
        min_needed = REGIME_WINDOW + 30  # Extra for indicator warmup
        if len(df) < min_needed + REGIME_HORIZON:
            print(f"  ⚠️ {ticker}: Skipped (only {len(df)} days, need {min_needed + REGIME_HORIZON})")
            continue
        
        ticker_samples = 0
        errors = 0
        
        # Start after warmup period
        for i in range(min_needed, len(df) - REGIME_HORIZON, 5):
            # Calculate forward return
            future_price = df.iloc[i + REGIME_HORIZON]['Close']
            current_price = df.iloc[i]['Close']
            forward_return = (future_price - current_price) / current_price
            
            # Label
            if forward_return > REGIME_THRESHOLD:
                label = 2  # BUY
            elif forward_return < -REGIME_THRESHOLD:
                label = 0  # SELL
            else:
                label = 1  # HOLD
            
            try:
                # Generate 20-channel regime-aware GASF
                gasf_img = generate_regime_aware_gasf(
                    ticker,
                    df.iloc[:i],  # Full history for context
                    spy_df.iloc[:i],
                    sector_data,
                    REGIME_WINDOW,
                    REGIME_IMAGE_SIZE
                )
                
                regime_aware_dataset.append({
                    'image': gasf_img,
                    'label': label,
                    'ticker': ticker,
                    'date': df.index[i],
                    'return': forward_return
                })
                ticker_samples += 1
            except Exception as e:
                errors += 1
                if errors == 1:
                    print(f"  ⚠️ {ticker}: Error: {str(e)[:80]}")
        
        if ticker_samples > 0:
            print(f"  ✓ {ticker}: {ticker_samples} regime-aware GASF images")
        elif errors > 0:
            print(f"  ✗ {ticker}: Failed ({errors} errors)")
    
    if len(regime_aware_dataset) > 0:
        print(f"\n✅ Generated {len(regime_aware_dataset)} regime-aware GASF images")
        print(f"   20 channels × {REGIME_IMAGE_SIZE}×{REGIME_IMAGE_SIZE} = {20*REGIME_IMAGE_SIZE*REGIME_IMAGE_SIZE:,} values per sample")
        print(f"\n🧠 INTELLIGENCE ADDED:")
        print(f"   ✓ Knows market regime (bull/bear/sideways)")
        print(f"   ✓ Sees sector rotation (hot vs cold sectors)")
        print(f"   ✓ Detects relative strength (underperformers = buy opps)")
        print(f"   ✓ Avoids chasing pumps (recent 5-day gain > 10%)")
        print(f"   ✓ Catches dips (down 5%+ from high + RSI < 40)")
        print(f"   ✓ Understands market beta (SPY correlation)")
    else:
        print(f"\n❌ No regime-aware GASF images generated!")

In [ ]:
# Show regime-aware class distribution
if len(regime_aware_dataset) == 0:
    print("\n⚠️ No regime-aware GASF images. Check errors above.")
else:
    labels_regime = [x['label'] for x in regime_aware_dataset]
    print(f"\n📊 Regime-Aware Dataset Class Distribution:")
    print(f"   SELL (0): {labels_regime.count(0)} ({labels_regime.count(0)/len(labels_regime)*100:.1f}%)")
    print(f"   HOLD (1): {labels_regime.count(1)} ({labels_regime.count(1)/len(labels_regime)*100:.1f}%)")
    print(f"   BUY (2): {labels_regime.count(2)} ({labels_regime.count(2)/len(labels_regime)*100:.1f}%)")
    print(f"\n✅ Regime-aware dataset ready: {len(regime_aware_dataset)} samples")
    print(f"   Memory: ~{len(regime_aware_dataset) * 20 * 60 * 60 * 4 / 1e6:.1f} MB")
    
    # Show context intelligence
    print(f"\n🌍 CONTEXT INTELLIGENCE:")
    print(f"   The CNN now sees each chart in CONTEXT:")
    print(f"   - Is the market in bull/bear/sideways mode?")
    print(f"   - Is this sector hot or cold?")
    print(f"   - Is this ticker underperforming its sector? (buying opp)")
    print(f"   - Did it pump recently? (avoid chasing)")
    print(f"   - Is it dipping with low RSI? (catch reversal)")
    print(f"   - How correlated is it with SPY? (market beta)")
    print(f"\n   This is how SMART traders think - not just chart patterns!")

In [ ]:
# Regime-aware AlphaGo network
class RegimeAwareAlphaGoNet(nn.Module):
    """
    Regime-aware dual-head network with:
    - 20-channel input (OHLCV + Indicators + Context + Regime)
    - Channel attention (learns importance of each context signal)
    - Spatial attention (focuses on critical time periods)
    - Multi-head output (action + confidence + regime-specific advice)
    """
    def __init__(self, num_actions=3, input_channels=20):
        super(RegimeAwareAlphaGoNet, self).__init__()
        
        # Channel attention: Learn which context matters most
        self.channel_attention = nn.Sequential(
            nn.Linear(input_channels, input_channels * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(input_channels * 2, input_channels),
            nn.Sigmoid()
        )
        
        # Modified ResNet-18 for 20 channels
        resnet = models.resnet18(pretrained=False)
        
        # First conv: 20 channels → 64 features
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Initialize with ImageNet weights (replicate for extra channels)
        resnet_pretrained = models.resnet18(pretrained=True)
        with torch.no_grad():
            weight = resnet_pretrained.conv1.weight
            # Replicate RGB pattern across all 20 channels
            for i in range(input_channels):
                self.conv1.weight[:, i:i+1] = weight[:, i % 3:i % 3 + 1]
        
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet blocks
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        self.avgpool = resnet.avgpool
        
        # POLICY HEAD (deeper + regime-aware)
        self.policy_fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_actions)
        )
        
        # VALUE HEAD (deeper)
        self.value_fc = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 1),
            nn.Tanh()
        )
        
        # CONFIDENCE HEAD (how sure are we?)
        self.confidence_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()  # [0, 1] confidence
        )
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        # Channel attention: Weight each channel (learns which context matters)
        channel_weights = self.channel_attention(x.mean(dim=[2, 3]))  # (B, C)
        channel_weights = channel_weights.view(B, C, 1, 1)
        x = x * channel_weights
        
        # Shared backbone
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        features = torch.flatten(x, 1)
        
        # Triple outputs
        policy_logits = self.policy_fc(features)
        value = self.value_fc(features)
        confidence = self.confidence_fc(features)
        
        return policy_logits, value, confidence

print("✅ Regime-aware AlphaGo network with triple output defined")
print("   Features: 20-channel input, channel attention, confidence head")
print("   Outputs: Policy (action) + Value (expected return) + Confidence (how sure)")

In [ ]:
# Initialize regime-aware model
regime_aware_model = RegimeAwareAlphaGoNet(num_actions=3, input_channels=20)
regime_aware_model = regime_aware_model.to(device)

print(f"🌍 Regime-Aware AlphaGo Trading Network")
print(f"   Device: {device}")
print(f"   Input: 20-channel GASF")
print(f"     - OHLCV (5)")
print(f"     - Technical Indicators (8): SMA, EMA, BB, VWAP, MACD, RSI, ATR")
print(f"     - Context (6): SPY corr, Sector corr, Rel strength, Vol surge, Pump, Dip")
print(f"     - Regime (1): Bull/Bear/Sideways")
print(f"   Image size: {REGIME_IMAGE_SIZE}×{REGIME_IMAGE_SIZE}")
print(f"   Channel attention: Learns which context matters most")
print(f"   Policy head: 3-class (SELL/HOLD/BUY)")
print(f"   Value head: Expected return regression")
print(f"   Confidence head: How sure is the model?")
print(f"   Parameters: {sum(p.numel() for p in regime_aware_model.parameters()):,}")
print(f"\n💡 This model THINKS like a professional trader!")
print(f"   - Sees market regime")
print(f"   - Knows sector rotation")
print(f"   - Avoids chasing pumps")
print(f"   - Catches dips before reversal")
print(f"   - Understands relative strength")

In [ ]:
# Prepare regime-aware dataset and loaders
if len(regime_aware_dataset) > 0:
    split_idx_regime = int(len(regime_aware_dataset) * 0.8)
    train_data_regime = regime_aware_dataset[:split_idx_regime]
    test_data_regime = regime_aware_dataset[split_idx_regime:]
    
    train_dataset_regime = GASFDataset(train_data_regime)
    test_dataset_regime = GASFDataset(test_data_regime)
    
    # Batch size 16 for larger 20-channel images
    train_loader_regime = DataLoader(train_dataset_regime, batch_size=16, shuffle=True, num_workers=2)
    test_loader_regime = DataLoader(test_dataset_regime, batch_size=16, shuffle=False, num_workers=2)
    
    print(f"📦 Regime-Aware DataLoaders:")
    print(f"   Train: {len(train_dataset_regime)} samples, {len(train_loader_regime)} batches")
    print(f"   Test: {len(test_dataset_regime)} samples, {len(test_loader_regime)} batches")
    print(f"   Batch size: 16 (20-channel 60×60 images)")
else:
    print("❌ No regime-aware dataset available. Generate it first.")

In [ ]:
# Training function for regime-aware model
def train_regime_aware_network(model, train_loader, test_loader, num_epochs=30, 
                                policy_weight=1.0, value_weight=0.5, confidence_weight=0.3):
    """
    Train triple-head network jointly.
    Loss = policy_weight * CrossEntropy + value_weight * MSE + confidence_weight * ConfidenceLoss
    
    Confidence loss: Model should be confident when correct, uncertain when wrong
    """
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    policy_criterion = nn.CrossEntropyLoss()
    value_criterion = nn.MSELoss()
    
    best_policy_acc = 0.0
    history = {'train_loss': [], 'test_policy_acc': [], 'test_value_mse': [], 'test_confidence': []}
    
    for epoch in range(num_epochs):
        # TRAINING
        model.train()
        running_policy_loss = 0.0
        running_value_loss = 0.0
        running_confidence_loss = 0.0
        
        for images, policy_labels, value_targets in train_loader:
            images = images.to(device)
            policy_labels = policy_labels.to(device)
            value_targets = value_targets.to(device)
            
            optimizer.zero_grad()
            
            # Forward
            policy_logits, value_preds, confidence = model(images)
            
            # Policy loss
            policy_loss = policy_criterion(policy_logits, policy_labels)
            
            # Value loss
            value_loss = value_criterion(value_preds, value_targets)
            
            # Confidence loss: Should be high when prediction is correct
            _, predicted = torch.max(policy_logits, 1)
            is_correct = (predicted == policy_labels).float().unsqueeze(1)
            # If correct, confidence should be high (close to 1)
            # If wrong, confidence should be low (close to 0)
            confidence_loss = F.binary_cross_entropy(confidence, is_correct)
            
            # Combined loss
            total_loss = (policy_weight * policy_loss + 
                         value_weight * value_loss + 
                         confidence_weight * confidence_loss)
            
            # Backward
            total_loss.backward()
            optimizer.step()
            
            running_policy_loss += policy_loss.item()
            running_value_loss += value_loss.item()
            running_confidence_loss += confidence_loss.item()
        
        avg_policy_loss = running_policy_loss / len(train_loader)
        avg_value_loss = running_value_loss / len(train_loader)
        avg_confidence_loss = running_confidence_loss / len(train_loader)
        avg_total_loss = (policy_weight * avg_policy_loss + 
                         value_weight * avg_value_loss + 
                         confidence_weight * avg_confidence_loss)
        history['train_loss'].append(avg_total_loss)
        
        # VALIDATION
        model.eval()
        policy_correct = 0
        policy_total = 0
        value_mse_sum = 0.0
        confidence_sum = 0.0
        
        with torch.no_grad():
            for images, policy_labels, value_targets in test_loader:
                images = images.to(device)
                policy_labels = policy_labels.to(device)
                value_targets = value_targets.to(device)
                
                policy_logits, value_preds, confidence = model(images)
                
                # Policy accuracy
                _, predicted = torch.max(policy_logits, 1)
                policy_total += policy_labels.size(0)
                policy_correct += (predicted == policy_labels).sum().item()
                
                # Value MSE
                value_mse_sum += value_criterion(value_preds, value_targets).item()
                
                # Average confidence
                confidence_sum += confidence.mean().item()
        
        policy_acc = 100 * policy_correct / policy_total
        value_mse = value_mse_sum / len(test_loader)
        avg_confidence = confidence_sum / len(test_loader)
        history['test_policy_acc'].append(policy_acc)
        history['test_value_mse'].append(value_mse)
        history['test_confidence'].append(avg_confidence)
        
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Loss: {avg_total_loss:.4f} (Policy: {avg_policy_loss:.4f}, Value: {avg_value_loss:.4f}, Conf: {avg_confidence_loss:.4f})")
        print(f"  Test - Policy Acc: {policy_acc:.2f}%, Value MSE: {value_mse:.4f}, Confidence: {avg_confidence:.3f}")
        
        # Save best
        if policy_acc > best_policy_acc:
            best_policy_acc = policy_acc
            torch.save(model.state_dict(), 'best_regime_aware_model.pth')
            print(f"  ✅ Best model saved (acc={best_policy_acc:.2f}%)")
        
        scheduler.step()
    
    return history, best_policy_acc

print("✅ Regime-aware training function defined")
print("   Triple loss: Policy + Value + Confidence")
print("   Confidence head learns: Be certain when correct, uncertain when wrong")

In [ ]:
# Train regime-aware model
if len(regime_aware_dataset) == 0:
    print("❌ No regime-aware dataset. Generate it first (run previous cells).")
elif 'regime_aware_model' not in locals():
    print("❌ Regime-aware model not initialized. Run the model initialization cell first.")
elif 'train_loader_regime' not in locals():
    print("❌ DataLoaders not created. Run the dataloader cell first.")
else:
    print("\n🌍 TRAINING REGIME-AWARE ALPHAGO NETWORK...")
    print("="*70)
    print("20 Channels: OHLCV + Indicators + CONTEXT + Regime")
    print("Context Intelligence:")
    print("  ✓ SPY correlation (market beta)")
    print("  ✓ Sector correlation (sector rotation)")
    print("  ✓ Relative strength (underperformers = buy opps)")
    print("  ✓ Volume surge (unusual activity)")
    print("  ✓ Pump detector (avoid chasing)")
    print("  ✓ Dip detector (catch reversals)")
    print("  ✓ Regime awareness (bull/bear/sideways)")
    print("="*70 + "\n")
    
    history_regime, best_acc_regime = train_regime_aware_network(
        regime_aware_model,
        train_loader_regime,
        test_loader_regime,
        num_epochs=30,  # More epochs for complex context model
        policy_weight=1.0,
        value_weight=0.5,
        confidence_weight=0.3
    )
    
    print(f"\n🏆 REGIME-AWARE MODEL RESULTS:")
    print(f"   Best Policy Accuracy: {best_acc_regime:.2f}%")
    print(f"   Baseline (5 channels): 59.71%")
    print(f"   Enhanced (13 channels): 52.46%")
    print(f"   Regime-Aware (20 channels): {best_acc_regime:.2f}%")
    print(f"   Improvement vs Enhanced: {best_acc_regime - 52.46:+.2f}%")
    
    if best_acc_regime > 70:
        print(f"\n🚀 BREAKTHROUGH! Regime-aware model > 70% accuracy")
        print(f"   Context intelligence is working!")
    elif best_acc_regime > 65:
        print(f"\n✅ TARGET ACHIEVED! Regime-aware model > 65% accuracy")
    elif best_acc_regime > 59.71:
        print(f"\n📈 IMPROVEMENT! Beats baseline")
    else:
        print(f"\n⚠️ Needs tuning. Try more epochs or different context features")

---
## 🔬 RESEARCH NEXT: Perplexity Questions for Further Optimization

Copy these questions to Perplexity Pro to optimize the regime-aware system:

**1. Market Regime Detection:**
```
What are the most accurate methods for detecting market regimes (bull/bear/sideways) in stock trading?
Compare: HMM (Hidden Markov Models), Gaussian Mixture Models, and simple trend-based approaches.
Which works best for daily stock data with 20-60 day windows?
Provide Python code for optimal regime detection.
```

**2. Sector Rotation Strategies:**
```
How do professional traders use sector rotation analysis to find buying opportunities?
Explain:
- How to detect when a sector is hot vs cold
- How to identify underperforming stocks in hot sectors (mean reversion plays)
- How to avoid overperforming stocks in cold sectors (false signals)
- Optimal lookback windows for sector relative strength
Provide Python implementation using sector ETFs (XLK, XLF, XLE, etc.)
```

**3. Pump & Dip Detection:**
```
What are the most reliable indicators for detecting:
- Stock pumps (to avoid chasing)
- Buying-the-dip opportunities (undervalued dips vs falling knives)

Compare indicators:
- RSI divergence
- Volume surge patterns
- Rate of change vs historical volatility
- Williams %R
- Money Flow Index

Provide optimal thresholds and Python code for pump/dip detection.
```

**4. Context-Aware CNNs:**
```
In computer vision, how do researchers add contextual information to CNNs?
Examples:
- Multi-modal CNNs (combining image + metadata)
- Conditioning on external variables
- Graph Neural Networks for relational context

How can we apply these techniques to financial CNNs where context = market regime, sector rotation, correlation?
Provide PyTorch implementation of context-aware CNN architecture.
```

**5. Confidence Calibration:**
```
How to train neural networks to output well-calibrated confidence scores?
The model should be:
- Highly confident when correct
- Low confidence when uncertain
- Avoid overconfidence

Compare methods:
- Temperature scaling
- Platt scaling  
- Mixup training
- Label smoothing
- Focal loss

Provide PyTorch code for training confidence-calibrated CNNs.
```

---
## PHASE 3: AlphaGo Dual Network Architecture

In [ ]:
class AlphaGoTradingNet(nn.Module):
    """
    Dual-head architecture inspired by AlphaGo.
    
    - Shared ResNet-18 backbone: Extract visual patterns
    - Policy Head: BUY/HOLD/SELL (classification)
    - Value Head: Expected return (regression)
    """
    def __init__(self, num_actions=3, input_channels=5):
        super(AlphaGoTradingNet, self).__init__()
        
        # Shared backbone: ResNet-18
        resnet = models.resnet18(pretrained=False)
        
        # Modify first conv for 5-channel input (OHLCV)
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Optionally initialize from ImageNet weights
        resnet_pretrained = models.resnet18(pretrained=True)
        with torch.no_grad():
            # Copy RGB channels
            self.conv1.weight[:, :3] = resnet_pretrained.conv1.weight
            # Duplicate for extra channels
            self.conv1.weight[:, 3:] = resnet_pretrained.conv1.weight[:, :2]
        
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet blocks
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4
        
        self.avgpool = resnet.avgpool
        
        # POLICY HEAD (Action)
        self.policy_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_actions)
        )
        
        # VALUE HEAD (Expected return)
        self.value_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Tanh()  # [-1, 1] range
        )
    
    def forward(self, x):
        # Shared backbone
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        features = torch.flatten(x, 1)
        
        # Dual outputs
        policy_logits = self.policy_fc(features)
        value = self.value_fc(features)
        
        return policy_logits, value

print("✅ AlphaGo dual network architecture defined")

In [ ]:
# Initialize model
model = AlphaGoTradingNet(num_actions=3, input_channels=5)
model = model.to(device)

print(f"🎮 AlphaGo Trading Network")
print(f"   Device: {device}")
print(f"   Input: 5-channel GASF (OHLCV)")
print(f"   Policy head: 3-class (SELL/HOLD/BUY)")
print(f"   Value head: Regression (expected return)")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

---
## PHASE 4: Dataset & DataLoader

In [ ]:
class GASFDataset(Dataset):
    """Dataset for GASF images with policy labels and value targets"""
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # GASF image (5, 64, 64)
        image = torch.FloatTensor(item['image'])
        
        # Policy label (0, 1, 2)
        policy_label = item['label']
        
        # Value target (normalized return [-1, 1])
        value_target = np.clip(item['return'] / 0.2, -1, 1)
        
        return image, policy_label, torch.FloatTensor([value_target])

print("✅ GASF Dataset class defined")

In [ ]:
# Split dataset (80/20, time-series aware)
split_idx = int(len(gasf_dataset) * 0.8)
train_data = gasf_dataset[:split_idx]
test_data = gasf_dataset[split_idx:]

train_dataset = GASFDataset(train_data)
test_dataset = GASFDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"📦 DataLoaders created:")
print(f"   Train: {len(train_dataset)} samples, {len(train_loader)} batches")
print(f"   Test: {len(test_dataset)} samples, {len(test_loader)} batches")

---
## PHASE 5: Training Function

In [ ]:
def train_alphago_network(model, train_loader, test_loader, num_epochs=20, 
                          policy_weight=1.0, value_weight=0.5):
    """
    Train dual-head network jointly.
    Loss = policy_weight * CrossEntropy + value_weight * MSE
    """
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    policy_criterion = nn.CrossEntropyLoss()
    value_criterion = nn.MSELoss()
    
    best_policy_acc = 0.0
    history = {'train_loss': [], 'test_policy_acc': [], 'test_value_mse': []}
    
    for epoch in range(num_epochs):
        # TRAINING
        model.train()
        running_policy_loss = 0.0
        running_value_loss = 0.0
        
        for images, policy_labels, value_targets in train_loader:
            images = images.to(device)
            policy_labels = policy_labels.to(device)
            value_targets = value_targets.to(device)
            
            optimizer.zero_grad()
            
            # Forward
            policy_logits, value_preds = model(images)
            
            # Combined loss
            policy_loss = policy_criterion(policy_logits, policy_labels)
            value_loss = value_criterion(value_preds, value_targets)
            total_loss = policy_weight * policy_loss + value_weight * value_loss
            
            # Backward
            total_loss.backward()
            optimizer.step()
            
            running_policy_loss += policy_loss.item()
            running_value_loss += value_loss.item()
        
        avg_policy_loss = running_policy_loss / len(train_loader)
        avg_value_loss = running_value_loss / len(train_loader)
        avg_total_loss = policy_weight * avg_policy_loss + value_weight * avg_value_loss
        history['train_loss'].append(avg_total_loss)
        
        # VALIDATION
        model.eval()
        policy_correct = 0
        policy_total = 0
        value_mse_sum = 0.0
        
        with torch.no_grad():
            for images, policy_labels, value_targets in test_loader:
                images = images.to(device)
                policy_labels = policy_labels.to(device)
                value_targets = value_targets.to(device)
                
                policy_logits, value_preds = model(images)
                
                # Policy accuracy
                _, predicted = torch.max(policy_logits, 1)
                policy_total += policy_labels.size(0)
                policy_correct += (predicted == policy_labels).sum().item()
                
                # Value MSE
                value_mse_sum += value_criterion(value_preds, value_targets).item()
        
        policy_acc = 100 * policy_correct / policy_total
        value_mse = value_mse_sum / len(test_loader)
        history['test_policy_acc'].append(policy_acc)
        history['test_value_mse'].append(value_mse)
        
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Loss: {avg_total_loss:.4f} (Policy: {avg_policy_loss:.4f}, Value: {avg_value_loss:.4f})")
        print(f"  Test - Policy Acc: {policy_acc:.2f}%, Value MSE: {value_mse:.4f}")
        
        # Save best
        if policy_acc > best_policy_acc:
            best_policy_acc = policy_acc
            torch.save(model.state_dict(), 'best_alphago_model.pth')
            print(f"  ✅ Best model saved (acc={best_policy_acc:.2f}%)")
        
        scheduler.step()
    
    return history, best_policy_acc

print("✅ Training function defined")

---
## PHASE 6: Train Model

In [ ]:
# Train
print("\n🚀 TRAINING ALPHAGO DUAL NETWORK...")
print("="*70)
print("Policy Network: Learns WHAT action to take")
print("Value Network: Learns EXPECTED outcome")
print("="*70 + "\n")

history, best_acc = train_alphago_network(
    model, 
    train_loader, 
    test_loader,
    num_epochs=20,
    policy_weight=1.0,
    value_weight=0.5
)

print(f"\n🏆 BEST POLICY ACCURACY: {best_acc:.2f}%")

---
## PHASE 7: Visualize Training

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(history['train_loss'])
axes[0].set_title('Training Loss (Combined)', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['test_policy_acc'], color='green')
axes[1].set_title('Policy Accuracy (What to do?)', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['test_value_mse'], color='red')
axes[2].set_title('Value MSE (Expected return)', fontsize=14)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('MSE')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("📊 Training curves saved as training_curves.png")

---
## PHASE 8: Download Trained Model

In [ ]:
# Save results
import json

results = {
    'best_policy_accuracy': best_acc,
    'final_policy_accuracy': history['test_policy_acc'][-1],
    'final_value_mse': history['test_value_mse'][-1],
    'num_train_samples': len(train_dataset),
    'num_test_samples': len(test_dataset),
    'num_tickers': len(data),
    'window_size': WINDOW_SIZE,
    'image_size': IMAGE_SIZE,
    'horizon': HORIZON,
    'threshold': THRESHOLD,
    'history': history
}

with open('training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to training_results.json")
print("✅ Model saved to best_alphago_model.pth")
print("\n📥 Download these files:")
print("   1. best_alphago_model.pth (model weights)")
print("   2. training_results.json (metrics)")
print("   3. training_curves.png (visualization)")

In [ ]:
# Download files (Colab only)
try:
    from google.colab import files
    files.download('best_alphago_model.pth')
    files.download('training_results.json')
    files.download('training_curves.png')
    print("✅ Files downloaded successfully!")
except:
    print("ℹ️ Not running in Colab. Files saved locally.")

---
## PHASE 9: Test Inference

In [ ]:
# Load best model and test
model.load_state_dict(torch.load('best_alphago_model.pth'))
model.eval()

# Test on a sample
sample_image, sample_label, sample_value = test_dataset[50]
sample_image_batch = sample_image.unsqueeze(0).to(device)

with torch.no_grad():
    policy_logits, value_pred = model(sample_image_batch)
    policy_probs = F.softmax(policy_logits, dim=1)
    predicted_action = torch.argmax(policy_probs, dim=1).item()

action_names = ['SELL', 'HOLD', 'BUY']
print("\n🧪 Test Inference:")
print(f"   True Label: {action_names[sample_label]}")
print(f"   Predicted: {action_names[predicted_action]}")
print(f"   Policy Probs: SELL={policy_probs[0,0]:.3f}, HOLD={policy_probs[0,1]:.3f}, BUY={policy_probs[0,2]:.3f}")
print(f"   Expected Return: {value_pred.item():.4f} (normalized)")
print(f"   Actual Return: {sample_value.item():.4f} (normalized)")

if predicted_action == sample_label:
    print("   ✅ CORRECT PREDICTION")
else:
    print("   ❌ INCORRECT PREDICTION")

---
## 🎉 TRAINING COMPLETE!

### Next Steps:
1. Download `best_alphago_model.pth`
2. Download `training_results.json`
3. Integrate with numerical model (HistGB)
4. Create hybrid ensemble (40% visual + 60% numerical)
5. Backtest on historical data
6. Paper trade for 1 week
7. Deploy if > 65% win rate

### Expected Performance:
- **Policy Accuracy:** 65-68% (target)
- **Win Rate:** 67-70% (vs 61.7% baseline)
- **Improvement:** +5-8% win rate

**Status:** 🟢 Ready for integration!